<a href="https://colab.research.google.com/github/dundysm/Credit-Card-Fraud-Detection/blob/master/GPT_personal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hyperparameters
batch_size = 16 # Number of independent sequences to process in parallel
block_size = 32 # Maximum context length for predictions
max_iters = 5000 # Maximum number of iterations for training
eval_interval = 100 # Interval for evaluating the model's performance
learning_rate = 1e-3 # Learning rate for the optimizer
device = 'cuda' if torch.cuda.is_available() else 'cpu' # Use GPU if available, otherwise use CPU
eval_iters = 200 # Number of iterations for evaluating performance
n_embd = 64 # Dimension of the embedding layer
n_head = 4 # Number of heads in the multi-head attention mechanism
n_layer = 4 # Number of layers in the Transformer model
dropout = 0.0 # Dropout rate to prevent overfitting

torch.manual_seed(55)


In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


In [ ]:
# Open input file and read the text
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Get all unique characters in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# Create mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

# Encoder function: takes a string and outputs a list of integers
encode = lambda s: [stoi[c] for c in s] 

# Decoder function: takes a list of integers and outputs a string
decode = lambda l: ''.join([itos[i] for i in l])


In [ ]:
# Train and test data splits
# Encode the text into integers and convert it to a tensor
data = torch.tensor(encode(text), dtype=torch.long)

# Use 90% of the data for training and the remaining for validation
n = int(0.9*len(data)) 
train_data = data[:n]
val_data = data[n:]

# Function for generating small batches of data
def get_batch(split):
    # Choose train or validation data based on input split
    data = train_data if split == 'train' else val_data
    # Select random starting indices for each batch of data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # Extract the corresponding blocks of data and target values
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # Move the data to GPU if available
    x, y = x.to(device), y.to(device)
    return x, y





In [ ]:
@torch.no_grad()
def estimate_loss():
    """Evaluate the model on the validation set and return the mean loss."""
    out = {}
    model.eval()  # Set the model to evaluation mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()  # Set the model back to training mode
    return out

class Head(nn.Module):
    """One head of the self-attention mechanism."""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)  # Linear layer for the keys
        self.query = nn.Linear(n_embd, head_size, bias=False)  # Linear layer for the queries
        self.value = nn.Linear(n_embd, head_size, bias=False)  # Linear layer for the values
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))  # Lower triangle matrix for masking attention scores

        self.dropout = nn.Dropout(dropout)  # Dropout layer for regularization

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B, T, C)
        q = self.query(x) # (B, T, C)
        # Compute attention scores
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # Perform the weighted aggregation of the values
        v = self.value(x) # (B, T, C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        # create a list of `num_heads` attention heads, each with its own parameters
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        # linear layer to project the concatenated output of all heads to `n_embd` dimensions
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # concatenate the output of all heads along the last dimension
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        # project the concatenated output using the linear layer
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        # a simple feedforward neural network with two linear layers
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), # first linear layer
            nn.ReLU(), # ReLU activation function
            nn.Linear(4 * n_embd, n_embd), # second linear layer
            nn.Dropout(dropout), # dropout layer
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        # the size of each attention head
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size) # multi-head self-attention layer
        self.ffwd = FeedFoward(n_embd) # feedforward layer
        self.ln1 = nn.LayerNorm(n_embd) # layer normalization after self-attention
        self.ln2 = nn.LayerNorm(n_embd) # layer normalization after feedforward layer

    def forward(self, x):
        # add the residual connection before and after layer normalization
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [ ]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx



In [ ]:
# Create a Bigram Language Model
model = BigramLanguageModel()

# Move the model to the device specified in the `device` variable
m = model.to(device)

# Print the number of parameters in the model
print(f"{sum(p.numel() for p in m.parameters())/1e6:.2f} M parameters")

# Create a PyTorch optimizer using the AdamW optimizer with the specified learning rate
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Train the model for `max_iters` iterations
for iter in range(max_iters):
    
    # Every `eval_interval` iterations, evaluate the loss on the train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"Step {iter}: Train Loss {losses['train']:.4f}, Val Loss {losses['val']:.4f}")
    
    # Sample a batch of data for training
    xb, yb = get_batch('train')
    
    # Evaluate the loss on the current batch of data
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Generate text from the model
# Initialize a context tensor with shape (1, 1) and dtype long on the device specified in `device`
context = torch.zeros((1, 1), dtype=torch.long, device=device)


0.21 M parameters
Step 0: Train Loss 4.3559, Val Loss 4.3614
Step 100: Train Loss 2.6576, Val Loss 2.6614
Step 200: Train Loss 2.5089, Val Loss 2.5197
Step 300: Train Loss 2.4147, Val Loss 2.4264
Step 400: Train Loss 2.3600, Val Loss 2.3634
Step 500: Train Loss 2.3077, Val Loss 2.3188
Step 600: Train Loss 2.2741, Val Loss 2.2917
Step 700: Train Loss 2.2312, Val Loss 2.2460
Step 800: Train Loss 2.1822, Val Loss 2.2028
Step 900: Train Loss 2.1341, Val Loss 2.1766
Step 1000: Train Loss 2.0993, Val Loss 2.1427
Step 1100: Train Loss 2.0830, Val Loss 2.1247
Step 1200: Train Loss 2.0404, Val Loss 2.0914
Step 1300: Train Loss 2.0168, Val Loss 2.0703
Step 1400: Train Loss 2.0116, Val Loss 2.0603
Step 1500: Train Loss 1.9706, Val Loss 2.0532
Step 1600: Train Loss 1.9520, Val Loss 2.0279
Step 1700: Train Loss 1.9311, Val Loss 2.0157
Step 1800: Train Loss 1.9092, Val Loss 2.0137
Step 1900: Train Loss 1.8859, Val Loss 1.9896
Step 2000: Train Loss 1.8846, Val Loss 1.9859
Step 2100: Train Loss 1.8746

# OUTPUT

In [ ]:
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))



Thou doours head he will veak prosnakede, and other are and
Whom bost a miter with you, affairsts woulds she have him leave: scome cought oor cure ontranus you lookes of
To his doumn, fee that judgt curet of the paird with obter all vanton, much and all his pausion;
Wits pence thee on: your lords, it it words.

LUCIONE:
Her 'Nevila.
To Marcullanuddes matting, morts roud cort rick, we if
the prace! but now, well not, the could, Grafe it mastap.
So you, were and the duch!
'Clifford, at a strenchight I weld me the ears:
What't, thereus not. But turneto on Jo;
My speep thou happtimest extrant!

CORINIO:
Then the failt in your graclermen,
As he duke him onhraince to be such
since waw the duke womb dreamest give your sign some part therem't,
Romeo all the eirentle: but put parded city that druth;
Make more magadst me, sire? but yet,
AndOr take fortuned I have shope again'd me, untaked and thou must Can such
The truman Gest non, as cure is edvery. YeWtry lives
A vensis dissope, when we powou

In [ ]:
a